In [1]:
import urllib.request
import json
import pandas as pd
from pprint import pprint

with open('bechdel.csv') as f:
    bechdel_data = pd.read_csv(f,encoding='utf-8')
#bechdel_data
omdb_data = dict()
exceptions = list()

id_list = bechdel_data['imdb_id'].tolist()
#id_list = ['0250258','0389619','2614684','1022603','0275847','1449283','0450278','1572006']
totalmovies = len(id_list)
i = 0
for theid in id_list:
    i = i + 1
    id_string = 'tt'    
    needed_zeroes = 7 - len(str(theid))
    for j in range(0,needed_zeroes):
        id_string = id_string + '0'
    id_string = id_string + str(theid)
    omdburl = 'http://www.omdbapi.com/?i={0}&apikey=949d45c1'.format(id_string)
    try:
        omdbpage = urllib.request.urlopen(omdburl).read().decode('utf-8')
        omdb_data[theid] = json.loads(omdbpage,encoding='utf-8')
    except:
        exceptions.append(theid)
        pass
    if (i % 1000 == 0):
        print('Processing movie '+str(i)+' of '+str(totalmovies)+'...')
        
omdb_df = pd.DataFrame.from_dict(omdb_data,orient='index')
omdb_df.index.name = 'imdb_id'
#omdb_df
#bechdel_data
omdb_df.to_csv('omdb.csv',encoding='utf-8')
print('Done')

Processing movie 1000 of 7253...
Processing movie 2000 of 7253...
Processing movie 3000 of 7253...
Processing movie 4000 of 7253...
Processing movie 5000 of 7253...
Processing movie 6000 of 7253...
Processing movie 7000 of 7253...
Done


In [2]:
import numpy as np
#import pandas as pd
import time
omdb_df = pd.read_csv('omdb.csv',encoding='utf-8')

# Drop non-movie-related columns
omdb_df.drop(['Season','seriesID','Episode','totalSeasons'],axis=1,inplace=True)

# Rename columns to what we want them to be.
omdb_df=omdb_df.rename(columns = {'Actors':'actors','Writer':'writers','Response':'response','Language':'languages'})
omdb_df=omdb_df.rename(columns = {'Title':'imdbtitle','Rated':'mpaa_rating','imdbRating':'userscore','Type':'type'})
omdb_df=omdb_df.rename(columns = {'Poster':'posterlink','BoxOffice':'imdb_boxoffice','Country':'countries'})
omdb_df=omdb_df.rename(columns = {'Ratings':'all_ratings_list','Genre':'genres','Runtime':'runtime','Metascore':'metascore'})
omdb_df=omdb_df.rename(columns = {'Awards':'awards','Plot':'imdb_plot','DVD':'dvd_release_date','Director':'director'})
omdb_df=omdb_df.rename(columns = {'Website':'website','Year':'imdbyear','Released':'imdb_release_date','imdbID':'imdb_id_str'})
omdb_df=omdb_df.rename(columns = {'Production':'studio','imdbVotes':'nVotes'})

# Drop non-movies
omdb_df = omdb_df[omdb_df['type'] == 'movie']

# Fix runtimes to be integers
omdb_df['runtime'] = omdb_df['runtime'].str.slice(stop=-4).replace('',np.nan)
omdb_df['runtime'] = omdb_df['runtime'].apply(float)
omdb_df['runtime'] = omdb_df['runtime'].apply(lambda x: '{:.0f}'.format(x))



# Fix release dates to be datetime, formatted as yyyy-mm-dd
omdb_df['imdb_release_date'] = pd.to_datetime(omdb_df['imdb_release_date'],format="%d %b %Y", errors='coerce')

# Convert Release dateimte into new columns for year, month, and week
#omdb_df['imdb_release_year'] = omdb_df['imdb_release_date'].dt.strftime('%Y')
#omdb_df['imdb_release_month'] = omdb_df['imdb_release_date'].dt.strftime('%B')
#omdb_df['imdb_release_weekday'] = omdb_df['imdb_release_date'].dt.strftime('%A')

#omdb_df['imdb_release_week'] = omdb_df['imdb_release_date'].apply(lambda x:x.week)

# Take the imdbVotes formatted as string containing "N/A" and comma-delimited thousands, convert to float
omdb_df['nVotes'] = omdb_df['nVotes'].dropna().replace('N/A',np.nan).dropna().apply(lambda x:float(x.replace(',','')))

#imdb_df[['imdb_id','imdbtitle','mpaa_rating','type','imdb_release_date','imdb_release_year','imdb_release_month','imdb_release_week','imdb_release_weekday']]
omdb_df['nVotes'] = omdb_df['nVotes'].apply(lambda x: '{:,.0f}'.format(x))

# Take the Metascore formatted as string containing "N/A", convert to float
# Also divide by 10 to make effect sizes more comparable
omdb_df['metascore'] = omdb_df['metascore'].dropna().replace('N/A',np.nan).dropna().apply(float)/10.

# Take the imdbRating formatted as string containing "N/A", convert to float
omdb_df['userscore'] = omdb_df['userscore'].dropna().replace('N/A',np.nan).dropna().apply(float)

# Create indicator variables for English and USA movies (English is at least one of its languages etc.) (0 = no, 1 =yes)
omdb_df['english'] = (omdb_df['languages'].dropna().str.contains('nglish')).astype(int)
omdb_df['usa'] = (omdb_df['countries'].dropna().str.contains('USA')).astype(int)

# Convert imdb_ID to int, set it as the index
omdb_df['imdb_id'] = omdb_df['imdb_id'].dropna().apply(int)
omdb_df = omdb_df.set_index('imdb_id')

#omdb_df.loc[3].transpose()
omdb_df.to_csv('omdb.csv',encoding='utf-8')
print('Done')

Done
